<a href="https://colab.research.google.com/github/MalshaPiumini/FYP2022/blob/test-01/FYP_2022March_mcsDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I M P O R T S**

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
!pip install unidecode
!pip install emoji --upgrade

In [96]:
# Importing Libraries
import pandas as pd
from bs4 import BeautifulSoup
from unidecode import unidecode
import numpy as np
import random
import re
import psycopg2 as ps
import string
import emoji
import joblib

In [97]:
#Testing emoji Library
print(emoji.emojize('Python es :pulgar_hacia_arriba:', language='es'))

Python es 👍


# **O P E N__D A T A S E T ✌**




In [98]:
# hateSpeechData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FYP Implementations/PSPD-Dataset-2022 - Sheet1.csv" )
hateSpeechData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FYP Implementations/PSPD-Dataset-2022 - balanced.csv" )

In [99]:
#Setting the column width
pd.set_option('display.max_colwidth', 100)

#Setting the column names
# hateSpeechData.columns = ['Text','Feeling', 'Emotion','Label']

hateSpeechData.columns = ['Comments','Label']

#Printing first 5 rows
hateSpeechData.head()

,Comments,Label
0,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak 🖕,Hate
1,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan 😣,Hate
2,pakaya ballek wage marila inna ekak dapan,Hate
3,eeya hutto idakin kolukara haththa 😅 blaakka,Hate
4,mona hutte sinduda me thu witharak,Hate


In [100]:
# check the shape of training and testing dataset
print(hateSpeechData.shape)

(4979, 2)


In [101]:
# Check null or NaN values are present in the training data and testing data
print(hateSpeechData.isnull().values.any())

True


In [102]:
# Removing NaN or null rows
hateSpeechData = hateSpeechData.dropna(axis="rows", how="any")
print(hateSpeechData.isnull().values.any())

False


In [103]:
print(hateSpeechData.shape)

(1854, 2)


In [104]:
#Converting all the data to lower case
hateSpeechData['Label'] = hateSpeechData['Label'].str.lower()
hateSpeechData.head()

,Comments,Label
0,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak 🖕,hate
1,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan 😣,hate
2,pakaya ballek wage marila inna ekak dapan,hate
3,eeya hutto idakin kolukara haththa 😅 blaakka,hate
4,mona hutte sinduda me thu witharak,hate


In [105]:
# Convert Labels to numeric
def convertLabels(inputDf):
  for i in range(len(inputDf)):
    inputDf = inputDf.replace('hate', '1')
    inputDf = inputDf.replace('neutral', '0')
  return inputDf 

hateSpeechData['Label']=hateSpeechData['Label'].apply(lambda x : convertLabels(x))
hateSpeechData.head(10)

,Comments,Label
0,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak 🖕,1
1,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan 😣,1
2,pakaya ballek wage marila inna ekak dapan,1
3,eeya hutto idakin kolukara haththa 😅 blaakka,1
4,mona hutte sinduda me thu witharak,1
5,wada na hutto balana gaman,1
6,nilame thoge putha wenna athi thoge laga inne thopita henema gahapiya kalakanni thirisan paula,1
7,camara ussan ena madya ballange athapaya kadiya yuthui,1
8,hena gahapan pake lawke,1
9,kari demellu kana palenna gaahanna one muslim ballo,1


In [106]:
print("Out of {} rows, {} are Cyberbullying, {} are Not-cyberbullying".format(len(hateSpeechData),
                                                       len(hateSpeechData[hateSpeechData['Label']=='1']),
                                                       len(hateSpeechData[hateSpeechData['Label']=='0'])))

Out of 1854 rows, 927 are Cyberbullying, 927 are Not-cyberbullying


# **C L E A N__D A T A**

In [107]:
import pandas as pd
import emoji as emj

EMOJIS = emj.UNICODE_EMOJI["en"]

#Move emojies to another column
def add_emoji_column(df):
  emoji_column = " "
  for emoji in EMOJIS:
    if emoji in df:
      emoji_column = emoji_column + emoji
  return emoji_column

In [ ]:
hateSpeechData['emoji']=hateSpeechData['Comments'].apply(lambda x : add_emoji_column(x))
hateSpeechData.head()

In [109]:
# Remove Emojies from the text
def remove_emojies(df):
  cleaned_emojies = ""
  for emoji in EMOJIS:
    if emoji in df:
      df = df.replace(emoji, " ")
  return df

In [ ]:
hateSpeechData['emoji_cleaned_text']=hateSpeechData['Comments'].apply(lambda x : remove_emojies(x))
hateSpeechData.head() 

***Sinhala converting to Singlish***

In [111]:
def sinhalaToSinglish(inputDf):
  for index in range(len(inputDf)):
      currTokenList = inputDf['emoji_cleaned_text'].values[index]
      singlishTokenList = []
      for token in currTokenList:
        singlishTokenList.append(unidecode(token))
        inputDf['emoji_cleaned_text'].values[index] = "".join(singlishTokenList);
  return inputDf

In [112]:
hateSpeechData['emoji_cleaned_text'] = hateSpeechData['emoji_cleaned_text'].str.lower()

In [113]:
sinhalaToSinglish(hateSpeechData)
hateSpeechData.head()

,Comments,Label,emoji,emoji_cleaned_text
0,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak 🖕,1,🖕,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak
1,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan 😣,1,😣,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan
2,pakaya ballek wage marila inna ekak dapan,1,,pakaya ballek wage marila inna ekak dapan
3,eeya hutto idakin kolukara haththa 😅 blaakka,1,😅,eeya hutto idakin kolukara haththa blaakka
4,mona hutte sinduda me thu witharak,1,,mona hutte sinduda me thu witharak


In [114]:
#Remove URLs
import re
def urlRemoving(inputDf):
  for i in range(len(inputDf)):
    currentPhase= inputDf['emoji_cleaned_text'].values[i]
    inputDf['emoji_cleaned_text'].values[i] = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', currentPhase)
  return inputDf 

In [115]:
urlRemoving(hateSpeechData)
hateSpeechData.head()

,Comments,Label,emoji,emoji_cleaned_text
0,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak 🖕,1,🖕,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak
1,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan 😣,1,😣,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan
2,pakaya ballek wage marila inna ekak dapan,1,,pakaya ballek wage marila inna ekak dapan
3,eeya hutto idakin kolukara haththa 😅 blaakka,1,😅,eeya hutto idakin kolukara haththa blaakka
4,mona hutte sinduda me thu witharak,1,,mona hutte sinduda me thu witharak


In [116]:
# Clean training data and testing data
# remove html tags
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

# remove line seperators
def remove_line_seperators(string):
    result=re.sub("\\n", "", string)
    return result

# remove repeated characters
def remove_repeated_characters(text):
    regex_pattern = re.compile(r'(.)\1+')
    clean_text = regex_pattern.sub(r'\1\1', text)
    return clean_text

# # seperate digit with text
# def separate_digit_text(text):
#     regex_patter = re.compile(r'([\d]+)([a-zA-Z]+)')
#     clean_text = regex_patter.sub(r'\1 \2', text)
#     return clean_text

# remove punctuations
def remove_punctuations(text):
    test_punc_removed = [(ch if ch not in string.punctuation else " ") for ch in text ]
    test_punc_removed_join = ''.join(test_punc_removed)
    return test_punc_removed_join

# remove extra space
def remove_extra_space(text):
    clean_text = ' '.join(text.strip().split())
    return clean_text

# clean all text
def cleantext(text):
    text_cleaned= remove_line_seperators(
            remove_extra_space(
                remove_punctuations(
                    remove_repeated_characters(remove_tags(text)            
                            )
                        )
                    )
                )
    # text_cleaned=separate_digit_text(text_cleaned)
    return text_cleaned

In [117]:
hateSpeechData['comments_fixed']=hateSpeechData['emoji_cleaned_text'].apply(lambda cw : cleantext(cw))
hateSpeechData.head()

,Comments,Label,emoji,emoji_cleaned_text,comments_fixed
0,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak 🖕,1,🖕,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak,mun kata aran kiyana katha balapanko nikn puke talk danna epa hutto ekath kalawak
1,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan 😣,1,😣,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan,ammata hukapu gon huththige putha samajeta adarshawath deyak karapan
2,pakaya ballek wage marila inna ekak dapan,1,,pakaya ballek wage marila inna ekak dapan,pakaya ballek wage marila inna ekak dapan
3,eeya hutto idakin kolukara haththa 😅 blaakka,1,😅,eeya hutto idakin kolukara haththa blaakka,eeya hutto idakin kolukara haththa blaakka
4,mona hutte sinduda me thu witharak,1,,mona hutte sinduda me thu witharak,mona hutte sinduda me thu witharak


# **T O K E N I Z E**

In [118]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

In [ ]:
#Tokenize training data
hateSpeechData['body_text_tokenized'] = hateSpeechData['comments_fixed'].apply(lambda x: tokenize(x.lower()))
hateSpeechData.head()

# **S T O P__W O R D__R E M O V A L**

In [120]:
# function to count the total tokens
def getTotalTokens(token_set):
    count = 0
    for tSet in token_set:
        count += len(tSet)
    return count

In [121]:
# function to count the total unique tokens
def getTotalUniqueTokens(token_set):
    count = 0
    for tSet in token_set:
        uniqueTokens = set(tSet)
        count += len(uniqueTokens)
    return count

In [122]:
#-------------------------COMPARE REMOVING AND WITHOUT REMOVING----------------------------
#Remove Stop words
stopword = ['dha', 'ya', 'mea', 'ea', 'haa', 'ma', 'bawa', 'nam', 'dhe', 'shdhaha', 'yi',
                                'shaha', 'lesa', 'hoo', 'aetha', 'nisa', 'meama', 'wa', 'gaena', 'wisin', 'wita',
                                'ema', 'thula', 'kota', 'yuthu', 'washayen', 'no', 'pilibadha', 'anuwa', 'eheth',
                                'pasu', 'eak', 'mean', 'sita', 'neatha', 'weani', 'nea', 'wada', 'bohoa', 'etha',
                                'naehea', 'thawath', 'bawata', 'pamanak', 'yam', 'samaga', 'matha', 'sae', 'namuth',
                                'peara', 'maha', 'weatha', 'pamana', 'samaga', 'nowea', 'magin', 'enam', 'weanuwen',
                                'yatathea', 'saema', 'pasuwa', 'thamayi', 'dhakwa', 'nowa', 'siyalu', 'yaeyi',
                                'aethaem', 'yanu', 'waenuwen', 'haema', 'pinisa', 'patan', 'sahitha', 'hari',
                                'aethulu', 'wagae', 'paridhi', 'pawa', 'thawa', 'samaharu', 'samahara', 'eath',
                                'wena', 'baewin', 'pahala', 'wadaath', 'haera', 'pita', 'eweni', 'aa', 'ehala',
                                'thulin', 'ekka', 'aadhi', 'mulu', 'sambandhayean', 'peawathi', 'oya', 'wath',
                                'pamani', 'wen', 'pura', 'ona', 'epa', 'ewita', 'hewath', 'yanuwen', 'meawaeni',
                                'ow', 'ethi', 'tika', 'ihatha', 'kara', 'kisiyam', 'aeyi', 'kisima', 'ara', 'heyin',
                                'kisidhu', 'hata', 'anek', 'idhiri', 'thek', 'waenuwata', 'haetiyata', 'yata',
                                'dhaeyi', 'mokadha', 'maghin', 'baew', 'an', 'ona', 'laga', 'uth', 'haraha',
                                'asala', 'baehaera', 'hoth', 'ekthara', 'uda', 'athurin', 'klhi', 'ayuru', 'noyek',
                                'atharin', 'ammea', 'aethuluwa', 'hariyatama', 'ethana', 'ehaata', 'kawara',
                                'wanaahi', 'osseama', 'naethinam', 'udhesa', 'naedha', 'baehae', 'misa', 'haeti',
                                'wata', 'bae', 'etharam', 'thawadhuratath', 'thawa', 'thuru', 'idhiriyae',
                                'edhiriyae', 'hariyata', 'ebadhu', 'naethdha', 'naedhdha', 'anea', 'mona',
                                'ethamath', 'ithamath', 'ethakota', 'aneakuth', 'awata', 'aetha', 'meabadhu',
                                'lath', 'thulata', 'kohomadha', 'kumana', 'lesin', 'badhu', 'oonaema', 'sahagatha',
                                'lagata', 'wethata', 'waenidhaa', 'onea', 'passea', 'thissea', 'saeti', 'baegin',
                                'sesu',
                                'bohoma', 'kotharam', 'aethath', 'yanuwen', 'koyi', 'upa', 'yae', 'kotharam', 'o',
                                'anith', 'onna', 'naethahoth', 'witharak', 'kawdha', 'hugak', 'naetheyi', 'pitatha',
                                'saema', 'nisama', 'passea', 'seaka', 'ewan', 'monawadha', 'samagama', 'thawadha',
                                'sadha', 'haebaeyi', 'atharin', 'lesata', 'dhigata', 'mandha', 'namuth', 'namudhu',
                                'eheanam', 'kepa', 'emagin', 'idhin', 'mokakdha', 'dhigea', 'witaka', 'meani',
                                'sambandhawa', 'nohoth', 'anna', 'monawadha', 'harima', 'naethinam', 'atharaata',
                                'pitathata', 'wan', 'yatin', 'wagea', 'nisath', 'pahalata', 'anya', 'eha', 'witama',
                                'vitama', 'noyekuth', 'withara',
                                'ihalin', 'iwatha', 'nomethiwa', 'hudeak', 'meaweni', 'waethin', 'weni', 'waewa',
                                'andhamin', 'meanma', 'was', 'kiipa', 'kochchra', 'methaenin', 'methena',
                                'idhiripita', 'purama', 'witaka', 'athishaya', 'dhepasa', 'peareadha', 'aethulata',
                                'atharea', 'yamkisi', 'aassithawa', 'thak', 'koa', 'uda', 'meatha', 'ehalata',
                                'dhenatamath', 'wenama', 'arabeaya', 'rahitha', 'pitin', 'paahea', 'aethinam',
                                'aeththam', 'atharam', 'kuwuruth', 'paridhdhen', 'watea', 'keabadhu', 'kothek',
                                'naththam', 'hindha', 'pasupasa', 'meha', 'ladheyi', 'paasa', 'pawa', 'perata',
                                'waediputa', 'aethin', 'ethek', 'vina', 'maedhin', 'idhiriyean', 'pitupasa',
                                'samaga', 'mae', 'la', 'maenawi', 'sampanna', 'athurean', 'meathram',
                                'obbata', 'nomilea', 'arabaya', 'abiyasa', 'idhiripasa', 'enamuth',
                                'mathin''issaraha', 'echchrara', 'ekko', 'behi', 'magin', 'magini', 'ketuwa',
                                'misak', 'eheamyi', 'hindha', 'idhin', 'lagin', 'asalin', 'dho''kelinma',
                                'virahitha', 'mathata', 'asalata', 'atharahura',
                                'mealagata', 'kethek', 'kola', 'ayyo', 'nomilea', 'ethanata', 'mulata', 'lu',
                                'samagi', 'che', 'nithi', 'wenidha', 'apoyi', 'ochchrata', 'aaubowan', 'huga',
                                'wanathuru', 'pamanata', 'pahathata', 'ewaka', 'pasakin', 'aethulea', 'yatathata',
                                'yabadha', 'nan',
                                'shik', 'shek', 'usi', 'anan', 'manan', 'ananmanan', 'ahaa', 'adhdharata',
                                'monayam', 'athana', 'aethule', 'aethulen', 'pahathin', 'virahithawa', 'aayi',
                                'enayin', 'samagin', 'obbehi', 'othana', 'ammo', 'sha', 'neathahoth', 'atharamaga',
                                'asabada', 'yabadhawa', 'seakwa', 'atharamaga', 'obben', 'adhdhara',
                                'achchrara', 'dhoho', 'chi', 'chik', 'adho', 'soshae', 'ethira', 'koyibata', 'ohea',
                                'oh', 'adhomeyi', 'uhu', 'uu', 'u', 'otharam', 'abiyasin', 'appachchiyea', 'ado',
                                'aane', 'ane', 'hindhama', 'ha', 'hapura', 'latha', 'ohe', 'an', 'ankisi', 'hurea',
                                'uuyi', 'adhdharin', 'embala', 'abiyasa', 'abiyasata',
                                'uuh', 'shi', 'sisi', 'koyibata', 'satapata', 'abiyesa', 'heleyiya', 'sata',
                                'satata'

                                ]

def remove_stopwords(tokenized_list):
    # text = [word for word in tokenized_list if word not in stopword]
    text =" ".join( [word for word in tokenized_list if word not in stopword])
    return text

In [ ]:
#Stop word removal testing data
hateSpeechData['body_text_nostop'] = hateSpeechData['body_text_tokenized'].apply(lambda x: remove_stopwords(x))
hateSpeechData.head(100)

# **V E C T O R I Z I N G**

# **Count Vectorizer**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

count_vect_sample = CountVectorizer()
X_counts_sample = count_vect_sample.fit_transform(hateSpeechData['body_text_nostop'])
print(X_counts_sample.shape)
print(count_vect_sample.get_feature_names())

In [ ]:
X_counts_sample

In [ ]:
X_counts_df = pd.DataFrame(X_counts_sample.toarray())
X_counts_df

In [ ]:
X_counts_df.columns = count_vect_sample.get_feature_names()
X_counts_df

# **N-Grams + Count Vectorizer**

***Using Unigrams***

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

unigram_vectorizer = CountVectorizer(ngram_range=(1,1))
X_countsUni = unigram_vectorizer.fit_transform(hateSpeechData['body_text_nostop'])
print(X_countsUni.shape)
print(unigram_vectorizer.get_feature_names())

In [ ]:
X_counts_dfUni = pd.DataFrame(X_countsUni.toarray())
X_counts_dfUni

In [ ]:
X_counts_dfUni.columns = unigram_vectorizer.get_feature_names()
X_counts_dfUni

***Using Bigrams***

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bigram_vectorizer = CountVectorizer(ngram_range=(2,2))
X_countsBi = bigram_vectorizer.fit_transform(hateSpeechData['body_text_nostop'])
print(X_countsBi.shape)
print(bigram_vectorizer.get_feature_names())

In [ ]:
X_counts_dfBi = pd.DataFrame(X_countsBi.toarray())
X_counts_dfBi

In [ ]:
X_counts_dfBi.columns = bigram_vectorizer.get_feature_names()
X_counts_dfBi

***Using Trigrams***

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

trigram_vectorizer = CountVectorizer(ngram_range=(3,3))
X_countsTri= trigram_vectorizer.fit_transform(hateSpeechData['body_text_nostop'])
print(X_countsTri.shape)
print(trigram_vectorizer.get_feature_names())

In [ ]:
X_counts_dfTri = pd.DataFrame(X_countsTri.toarray())
X_counts_dfTri

In [ ]:
X_counts_dfTri.columns = trigram_vectorizer.get_feature_names()
X_counts_dfTri

# ***Bag Of Words***

In [132]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer
BOWvectorizer = CountVectorizer()
singlish_bag_of_words_X = BOWvectorizer.fit_transform(list(hateSpeechData['body_text_nostop']))

In [ ]:
print(singlish_bag_of_words_X)

In [ ]:
singlish_bag_of_words_y = list(hateSpeechData.Label)
print(singlish_bag_of_words_y)

In [135]:
# encoding labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
singlish_bag_of_words_y = le.fit_transform(singlish_bag_of_words_y)

In [136]:
# train test split
from sklearn.model_selection import train_test_split

# singlish dataset
X_train_singlish, X_test_singlish, y_train_singlish, y_test_singlish = train_test_split(singlish_bag_of_words_X, singlish_bag_of_words_y, test_size=0.2, random_state=42)

***Logistic Regression Model***

In [ ]:
# Logistic Regression Model for singlish
from sklearn.linear_model import LogisticRegression
logreg_singlish = LogisticRegression()
logreg_singlish.fit(X_train_singlish, y_train_singlish)

In [138]:
singlish_prediction = logreg_singlish.predict(X_test_singlish)

In [ ]:
print("Accuracy for singlish dataset:", logreg_singlish.score(X_test_singlish, y_test_singlish))

In [ ]:
# precision, recall and fscore
from sklearn.metrics import precision_recall_fscore_support
p, r, f, s =  precision_recall_fscore_support(y_test_singlish, singlish_prediction, average='macro')
print("Precision for singlish dataset:", p)
print("Recall for singlish dataset:", r)
print("F score for singlish dataset:", f)

In [ ]:
# confusion matrix
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(logreg_singlish, X_test_singlish, y_test_singlish)

***Naive Bayes Classifier***

In [ ]:
from sklearn.naive_bayes import MultinomialNB
#Naive Bayes Classifier
clf = MultinomialNB()
clf.fit(X_train_singlish, y_train_singlish)

In [143]:
singlish_prediction = clf.predict(X_test_singlish)

In [ ]:
print("Accuracy for singlish dataset:", logreg_singlish.score(X_test_singlish, y_test_singlish))

In [ ]:
p, r, f, s =  precision_recall_fscore_support(y_test_singlish, singlish_prediction, average='macro')
print("Precision for singlish dataset:", p)
print("Recall for singlish dataset:", r)
print("F score for singlish dataset:", f)

In [ ]:
# confusion matrix
plot_confusion_matrix(logreg_singlish, X_test_singlish, y_test_singlish)